# NB 3
Now let's consider a mother's likelihood to return to work after having more children.

In [9]:
# Set Up
import pandas as pd
import numpy as np

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Regression
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from IPython.core.display import HTML
from statsmodels.sandbox.regression.gmm import IV2SLS

# Graphing
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,10)
import seaborn as sns

# Specialty Imports

For the IV regression I'm going to be using statsmodels implementation of IV/2-Stage Least Squares. Documentation can be found here https://www.statsmodels.org/stable/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html

# Loading Data

In [10]:
sample = pd.read_csv('sample.csv')

## Creating my feature matrix, sample3

In [11]:
sample3 = pd.DataFrame()

sample3['noHSm'] = [1 if i < 12 else 0 for i in sample['educm']]
sample3['noHSd'] = [1 if i < 12 else 0 for i in sample['educd']]

sample3['agem_squared'] = sample['agem'] ** 2
sample3['agefstm_squared'] = sample['agefstm'] ** 2

sample3['aged_squared'] = sample['aged'] ** 2
sample3['agefstd_squared'] = sample['agefstd'] ** 2

sample3 = pd.concat([sample3, sample[['educm', 'educd', 'agem', 'aged', 'agefstm', 'agefstd',
                                      'blackm', 'hispm', 'othracem']]], axis = 1)



sample3.head()

,noHSm,noHSd,agem_squared,agefstm_squared,aged_squared,agefstd_squared,educm,educd,agem,aged,agefstm,agefstd,blackm,hispm,othracem
0,0,0,729,324,1225,676,12,12,27,35,18,26,0,0,0
1,1,1,900,324,784,289,7,9,30,28,18,17,0,0,0
2,0,0,729,441,900,576,12,13,27,30,21,24,0,0,0
3,0,0,1225,729,1296,841,14,19,35,36,27,29,1,0,0
4,0,0,900,484,1156,676,14,18,30,34,22,26,0,0,0


Let's begin by creating some linear probability models with control variables for education and race.

In [12]:
def part3_lpm(features, outcome, intercept = True):
    
    
    model = sm.OLS(outcome ,sm.add_constant(features))
    model = model.fit()
    
    return print(model.summary())
    

$$ p(workedm_i) = \beta_0 + \beta_1morekids + \epsilon_i$$

In [13]:
mw1_features = sample[['morekids']]
mw1_outcome = sample[['workedm']]

model_w1 = part3_lpm(mw1_features, mw1_outcome)

                            OLS Regression Results                            
Dep. Variable:                workedm   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2896.
Date:                Mon, 25 May 2020   Prob (F-statistic):               0.00
Time:                        12:44:28   Log-Likelihood:            -1.6963e+05
No. Observations:              236459   AIC:                         3.393e+05
Df Residuals:                  236457   BIC:                         3.393e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5765      0.001    447.648      0.0

$$ p(workedm) = 
\beta_0 noHSm + 
\beta_1 noHSd +
\beta_2 agem^2 +
\beta_3 agefstm^2  +
\beta_4 aged^2 +
\beta_5 agefstd_^2 +
\beta_6 educm +
\beta_7 educd +
\beta_8 agem +
\beta_9 aged +
\beta_{10} agefstm +
\beta_{11} agefstd +
\beta_{12} blackm +
\beta_{13} hispm +
\beta_{14} othracem +
\beta_{15} morekids +
\epsilon_i $$

In [14]:
mw2_features = pd.concat([sample3, sample['morekids']], axis = 1 )

model_w2 = part3_lpm(mw2_features, mw1_outcome)

                            OLS Regression Results                            
Dep. Variable:                workedm   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     1044.
Date:                Mon, 25 May 2020   Prob (F-statistic):               0.00
Time:                        12:44:29   Log-Likelihood:            -1.6299e+05
No. Observations:              236459   AIC:                         3.260e+05
Df Residuals:                  236442   BIC:                         3.262e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.6719      0.079     

## Analysis 

Without controls, having more kids seems to decrease the likelihood of a mother returning to work by about 11%. When we implement the various controls in terms of age, race, and education we see the probability increases to 15%. The original model was most likely suffering from unobserved information (omitted variable bias). 


### Now let's start running the code to generate our models.
Below is a function I wrote to verify that the IV regression package is working as expected. Long story short, I'm making sure the theory matches my empirical outputs.

In [15]:
def IV_2SLS_show_all(dependent, exogenous, instrument):
    """
    Runs the first stag and reduced form models via OLS,
    then runs the 2SLS/IV regression via statsmodels.
    
    Input:
        - dependent: a (nx1) outcome variable
        - exogenous: a (nx1) exogenous variable
        - instrument: a (nx1) instrument to be used for the exogenous variable
    Output:
        prints 3 summary tables of the regression
        1 DataFrame featuring
            - pi: the coeffecient of the first stage
            - delta: the coeffecient of the reduced form
            - ratio: delta/pi
            - iv2sls: coeffecient of the instrument in IV regression
    """

    iv = IV2SLS(dependent, sm.add_constant(exogenous),  sm.add_constant(instrument)).fit()
    iv2sls_coef = iv.params['morekids']
    
    first_stage = sm.OLS(exogenous, sm.add_constant(instrument)).fit()
    pi = first_stage.params['samesex']

    reduced = sm.OLS(dependent, sm.add_constant(instrument)).fit()
    delta = reduced.params['samesex']

    print(f"The first stage regression has a parameter of {pi} for the instrument.")
    print(f"The reduced stage regression has a parameter of {delta} for the instrument.")
    print(f"The ratio of delta/pi is {delta/pi} which is approximately equal to {iv2sls_coef}")
    
    
    
IV_2SLS_show_all(dependent = sample[['workedm']], 
                 exogenous = sample[['morekids']], 
                 instrument = sample[['samesex']])

The first stage regression has a parameter of 0.06735206106611896 for the instrument.
The reduced stage regression has a parameter of -0.01024119941692419 for the instrument.
The ratio of delta/pi is -0.15205472935520845 which is approximately equal to -0.15205472919798976


In [16]:
exo_controls = pd.concat([sample3, sample['morekids']], axis = 1)
instrument_controls = pd.concat([sample3, sample['samesex']], axis = 1)


iv = IV2SLS(sample[['workedm']], sm.add_constant(exo_controls), sm.add_constant(instrument_controls)).fit()
first_stage = sm.OLS(sample[['morekids']], sm.add_constant(instrument_controls)).fit()
reduced = sm.OLS(sample[['workedm']], sm.add_constant( pd.concat([sample3, sample['samesex']], axis = 1))).fit()

print("The ratio of delta and pi with controls is " + str(reduced.params['samesex']/first_stage.params['samesex']))
print("The 2sls estimate is" + str(iv.params['morekids']))

The ratio of delta and pi with controls is -0.1372496070115189
The 2sls estimate is-0.1372496070110102


# The IV Model

In [19]:
iv.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:                workedm   R-squared:                       0.066
Model:                         IV2SLS   Adj. R-squared:                  0.066
Method:                     Two Stage   F-statistic:                     700.3
                        Least Squares   Prob (F-statistic):               0.00
Date:                Mon, 25 May 2020                                         
Time:                        12:44:39                                         
No. Observations:              236459                                         
Df Residuals:                  236442                                         
Df Model:                          16                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.6822      0.080      8.512      0.000       0.525       0.839
noHSm              -0.0092      0.004     -2.169      0.030      -0.017      -0.001
noHSd              -0.0116      0.004     -2.941      0.003      -0.019      -0.004
agem_squared        0.0001   9.74e-05      1.145      0.252   -7.95e-05       0.000
agefstm_squared     0.0001      0.000      1.200      0.230   -8.17e-05       0.000
aged_squared        0.0001   5.95e-05      1.998      0.046    2.25e-06       0.000
agefstd_squared -9.487e-06   6.62e-05     -0.143      0.886      -0.000       0.000
educm               0.0314      0.001     45.307      0.000       0.030       0.033
educd              -0.0093      0.001    -17.591      0.000      -0.010      -0.008
agem                0.0041      0.006      0.665      0.506      -0.008       0.016
aged                0.0043      0.004      0.997      0.319      -0.004       0.013
agefstm            -0.0263      0.005     -5.322      0.000      -0.036      -0.017
agefstd            -0.0159      0.004     -4.264      0.000      -0.023      -0.009
blackm              0.1902      0.005     38.429      0.000       0.181       0.200
hispm               0.0865      0.007     11.788      0.000       0.072       0.101
othracem            0.0958      0.006     15.684      0.000       0.084       0.108
morekids           -0.1372      0.029     -4.771      0.000      -0.194      -0.081
==============================================================================
Omnibus:                   964313.966   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30748.527
Skew:                          -0.116   Prob(JB):                         0.00
Kurtosis:                       1.249   Cond. No.                     1.31e+05
==============================================================================
"""

From the regression output above we see that having morekids, using if the parent's had same sex children as an instrument, decreases the probability of the mother returning to work by about 13%. While it may seem as if we only took the average of 11% and 15% from the table above, I hope that the process of determining exogeneity for the IV model was clear and that the 13% claim we are making is well-supported by theory and empirical result.